# Traits

## Defining Traits

* Traits are Rust’s take on interfaces or abstract base classes
* They define shared behavior that types can implement
* They enable polymorphism and code reuse

In [2]:
trait Shape {
    fn move_by(&mut self, dx: i32, dy: i32);
    fn area(&self) -> f64;
}

## Implementing Traits

* To implement a trait for a type, we use the `impl` keyword

In [3]:
#[derive(Debug, Copy, Clone)]
struct Point {
    x: i32,
    y: i32
}

#[derive(Debug, Copy, Clone)]
struct Circle {
    position: Point,
    radius: u32
}

#[derive(Debug, Copy, Clone)]
struct Rectangle {
    position: Point,
    width: u32,
    height: u32
}

impl Shape for Circle {
    fn move_by(&mut self, dx: i32, dy: i32) {
        self.position.x += dx;
        self.position.y += dy;
    }

    fn area(&self) -> f64 {
        std::f64::consts::PI * (self.radius as f64) * (self.radius as f64)
    }
}

impl Shape for Rectangle {

    fn move_by(&mut self, dx: i32, dy: i32) {
        self.position.x += dx;
        self.position.y += dy;
    }

    fn area(&self) -> f64 {
        (self.width * self.height) as f64
    }
}

* We can implement many traits for a type:

In [4]:
trait Drawable {
    fn draw(&self);
}

impl Drawable for Circle {
    fn draw(&self) {
        println!("Drawing the circle at {:?} with radius: {}", self.position, self.radius);
    }
}

impl Drawable for Rectangle {
    fn draw(&self) {
          println!("Drawing the rectangle at {:?} with width: {} and height: {}", self.position, self.width, self.height);
  }
}

fn main() {
    let mut c = Circle { position: Point { x: 0, y: 0 }, radius: 10 };
    let mut r = Rectangle { position: Point { x: 0, y: 0 }, width: 10, height: 20 };

    c.draw();
    r.draw();

    c.move_by(10, 20);
    r.move_by(10, 20);

    println!("--- after move:");

    c.draw();
    r.draw();
}

main();

Drawing the circle at Point { x: 0, y: 0 } with radius: 10
Drawing the rectangle at Point { x: 0, y: 0 } with width: 10 and height: 20
--- after move:
Drawing the circle at Point { x: 10, y: 20 } with radius: 10
Drawing the rectangle at Point { x: 10, y: 20 } with width: 10 and height: 20


## Trait Bounds

* Trait bounds specify that a generic can be any type that implements a trait. 
* They work like a contracts at compile time

### impl Trait Syntax

* The `impl Trait` syntax is a shorthand for a longer form, called a **trait bound**

In [5]:
fn draw_shape(shape: &impl Drawable) {
    shape.draw()
}

In [6]:
let mut circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
let mut rect = Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5};

draw_shape(&circle);
draw_shape(&rect);

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5


### Trait Bound Syntax

In [7]:
fn draw_shape<T: Drawable>(shape: &T) {
    shape.draw()
}

draw_shape(&circle);
draw_shape(&rect);

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5


### Many trait bounds

* We can specify multiple trait bounds on a generic type by using the `+` syntax

In [8]:
fn use_shape(shape: &mut (impl Shape + Drawable)) {
    shape.move_by(10, 15);
    shape.draw();
}

In [ ]:
fn use_shape<T: Shape + Drawable>(shape: &mut T) {
    shape.move_by(10, 15);
    shape.draw();
}

* We can use a `where` clause to specify trait bounds on generic types

In [9]:
fn use_shape<T>(shape: &mut T)
where 
    T: Shape + Drawable 
{
    shape.move_by(10, 15);
    shape.draw();
}

In [ ]:
use_shape(&mut circle);
use_shape(&mut rect);

### Returning Types that Implement Traits

In [10]:
fn create_drawable(id: &str) -> Box<dyn Drawable> {
    match id {
        "Circle" => Box::new(Circle{position: Point{x: 0, y: 0}, radius: 1}),
        "Rectangle" => Box::new(Rectangle{position: Point{x: 0, y: 0}, width: 2, height: 1}),
        _ => panic!("Unknown shape")
    }
}

let shape: Box<dyn Drawable> = create_drawable("Circle");
shape.draw();

Drawing the circle at Point { x: 0, y: 0 } with radius: 1


### Self in Traits

* `Self` is an alias for the type we’re implementing the trait for

In [11]:
trait Cloneable {
    fn clone(&self) -> Self; // the type of x.clone() is the same as the type of x, whatever that might be
}

* A trait that uses the `Self` type is incompatible with trait objects
* Trait objects require dynamic dispatch, but using `Self` in a trait implies static dispatch — the compiler must know the concrete type at compile time.


In [12]:
fn do_clone(prototype: &dyn Cloneable) -> Box<dyn Cloneable> {
    let cloned_object = prototype.clone();
    Box::new(cloned_object)
}

Error: the trait `Cloneable` is not dyn compatible

## Trait Objects

* A trait object allows to implement dynamic polymorphism in Rust

In [13]:
let circle = Circle{position: Point{x: 10, y: 20}, radius: 5};

let shape: dyn Shape = circle; // ERROR: Shape does not have a constant size

Error: mismatched types

Error: the size for values of type `dyn Shape` cannot be known at compilation time

### References to Trait

* A reference to trait type, like `Shape` is called **trait object**. Like any other reference, a trait object points to some value, it has a lifetime, and it can be either mut or shared.
* In memory, a trait object is a **fat pointer** consisting of a pointer to the value, plus a pointer to a table representing that value’s type (virtual table).



In [14]:
fn main() {
    let mut shape: &dyn Shape;
    
    let circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
    shape = &circle;
    println!("Shape area: {:.2}", shape.area());

    shape = &Rectangle{position: Point{x: 10, y: 20}, width: 10, height: 20};
    println!("Shape area: {:.2}", shape.area());
}

main();

Shape area: 78.54
Shape area: 200.00


### Boxes to Trait

* We can also use a `Box` to store a trait object
* A `Box<dyn Trait>` is a trait object that points to a value on the heap

In [15]:
let box_circle: Box<Circle> = Box::new(Circle{position: Point{x: 10, y: 20}, radius: 5});
let mut shape: Box<dyn Shape> = box_circle; // OK - Box<dyn Shape> is trait object
println!("Shape area: {:.2}", shape.area());

shape = Box::new(Rectangle{position: Point{x: 10, y: 20}, width: 10, height: 20});
println!("Shape area: {:.2}", shape.area());

Shape area: 78.54
Shape area: 200.00


## Dynamic Polymorphism in Rust

In [16]:
#[derive(Debug)]
struct Square {
    position: Point,
    side: u32
}   

impl Shape for Square {

    fn move_by(&mut self, dx: i32, dy: i32) {
        self.position.x += dx;
        self.position.y += dy;
    }

    fn area(&self) -> f64 {
        (self.side * self.side) as f64
    }
}

impl Drawable for Square {
    fn draw(&self) {
        println!("Drawing the square at {:?} with side: {}", self.position, self.side);
    }
}

fn main() {
    let mut shapes: Vec<Box<dyn Shape>> = vec![
        Box::new(Circle{position: Point{x: 10, y: 20}, radius: 5}),
        Box::new(Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5}),
        Box::new(Square{position: Point{x: 70, y: 80}, side: 7})
    ];

    for shape in shapes.iter_mut() {
        shape.move_by(5, 5);
        println!("Shape area: {:.2}", shape.area());
    }
}

main();

Shape area: 78.54
Shape area: 50.00
Shape area: 49.00


### Combining Traits

* Trait objects can only use one trait at a time, but you can work around this by creating a supertrait:

In [19]:
trait DrawableShape: Shape + Drawable {} // Trait that combines Shape and Drawable - a supertrait

impl<T> DrawableShape for T 
    where T: Shape + Drawable 
{} // Implement DrawableShape for any type that implements Shape and Drawable - blanket implementation

fn main() {
    let mut drawable_shapes: Vec<Box<dyn DrawableShape>> = vec![
        Box::new(Circle{position: Point{x: 10, y: 20}, radius: 5}),
        Box::new(Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5}),
        Box::new(Square{position: Point{x: 70, y: 80}, side: 7})
    ];

    for shape in drawable_shapes.iter_mut() {
        shape.move_by(10, 15);
        shape.draw();
    }
}

main();

Drawing the circle at Point { x: 20, y: 35 } with radius: 5
Drawing the rectangle at Point { x: 50, y: 65 } with width: 10 and height: 5
Drawing the square at Point { x: 80, y: 95 } with side: 7


## Orphan Rules

* The **orphan rule** prevent us from implementing a foreign trait on a foreign type
* **You are allowed to implement any trait on any type, as long as either the trait or the type is local to your crate**

## Extension traits

In [ ]:
// Extension trait for checking if a character is an emoji
trait IsEmoji {
    fn is_emoji(&self) -> bool;
}

// Implementation of the IsEmoji trait for the char type
impl IsEmoji for char {
    fn is_emoji(&self) -> bool {
        let c = *self as u32;
        (c >= 0x1F600 && c <= 0x1F64F) || (c >= 0x1F300 && c <= 0x1F5FF) || (c >= 0x1F680 && c <= 0x1F6FF) || (c >= 0x1F900 && c <= 0x1F9FF)
    }
}

fn main() {
    let c = '😀';
    println!("Is '{}' an emoji? {}", c, c.is_emoji());
    println!("Is '{}' an emoji? {}", '@', '@'.is_emoji());
}

main();

* `IsEmoji` is an *extension trait* - adds a method to an existing type

## Generic `impl` blocks

* A generic `impl` block for a trait allows you to implement a trait for any type that satisfies certain bounds. This is a powerful way to write reusable, composable behavior across many types.

In [ ]:
use std::fmt::Debug;

trait DrawInDebug {
    fn draw_dbg(&self);
}

impl<T: Debug + Drawable> DrawInDebug for T {
    fn draw_dbg(&self) {
        println!("Drawing a {:?}", self);
    }
}

In [ ]:
circle.draw_dbg();
rect.draw_dbg();

## Associated Types

* Associated types are a way to associate a type placeholder with a trait
* Each type that implements the trait can specify its own concrete type for the associated type

In [20]:
trait Iterator {
    type Item; // associated type
    fn next(&mut self) -> Option<Self::Item>;
}

struct Counter {
    value: u32,
    sentinel: u32
}

impl Counter {
    fn new(sentinel: u32) -> Counter {
        Counter{value: 0, sentinel}
    }
}

impl Iterator for Counter {
    type Item = u32; // definition of associated type for Counter

    fn next(&mut self) -> Option<Self::Item> {
        if self.value < self.sentinel {
            let result = self.value;
            self.value += 1;
            Some(result)
        } else {
            None
        }
    }
}

fn main() {
    let mut counter = Counter::new(5);

    while let Some(value) = counter.next() {
        println!("{}", value);
    }
}

main();

0
1
2
3
4


* Generic code can use associated types:

In [ ]:
fn collect_into_vector<I: Iterator>(mut iter: I) -> Vec<I::Item> // I::Item is the associated type of the Iterator
{
    let mut results = Vec::new();

    while let Some(value) = iter.next() {
        results.push(value);
    }

    results
}

fn main() {    
    let numbers = collect_into_vector(Counter::new(5));
    println!("Collected numbers: {:?}", numbers);
}

main();

## Subtraits

* A trait can be an extension of another trait

In [21]:
trait BidirectionalIterator: Iterator {
    fn prev(&mut self) -> Option<Self::Item>;
}

impl BidirectionalIterator for Counter {
    fn prev(&mut self) -> Option<Self::Item> {
        if self.value > 0 {
            let result = self.value;
            self.value -= 1;
            Some(self.value)
        } else {
            None
        }
    }
}

fn main() {
    let mut counter = Counter::new(5);

    println!("Forward iteration:");
    while let Some(value) = counter.next() {
        println!("Counter value: {}", value);
    }

    println!("\nBackward iteration:");
    while let Some(value) = counter.prev() {
        println!("Counter value: {}", value);
    }
}

main();

Forward iteration:
Counter value: 0
Counter value: 1
Counter value: 2
Counter value: 3
Counter value: 4

Backward iteration:
Counter value: 4
Counter value: 3
Counter value: 2
Counter value: 1
Counter value: 0


## Static Methods in Traits

* Traits can have static methods

In [22]:
trait StringSet {
    /// Creates a new, empty set. - static method
    fn new() -> Self;

    /// Creates a set from a slice of strings. - static method
    fn from_slice(values: &[&str]) -> Self;

    /// Adds a string to the set.
    fn add(&mut self, value: &str);

    /// Removes a string from the set.
    fn contains(&self, value: &str) -> bool;
}

struct HashSet {
    data: std::collections::HashSet<String>
}

impl StringSet for HashSet {
    fn new() -> Self {
        HashSet{data: std::collections::HashSet::new()}
    }

    fn from_slice(values: &[&str]) -> Self {
        let mut set = HashSet::new();
        for value in values {
            set.add(value);
        }
        set
    }

    fn add(&mut self, value: &str) {
        self.data.insert(value.to_string());
    }

    fn contains(&self, value: &str) -> bool {
        self.data.contains(value)
    }
}

fn main() {
    println!("Creating set:");
    let mut set = HashSet::new();
    set.add("apple");
    set.add("banana");
    set.add("orange");

    println!("Set contains 'apple': {}", set.contains("apple"));
    println!("Set contains 'grape': {}", set.contains("grape"));

    println!("\nCreating set from slice:");

    let set = HashSet::from_slice(&["apple", "banana", "orange"]);
    println!("Set contains 'apple': {}", set.contains("apple"));
    println!("Set contains 'grape': {}", set.contains("grape"));
}

main();

Creating set:
Set contains 'apple': true
Set contains 'grape': false

Creating set from slice:
Set contains 'apple': true
Set contains 'grape': false


## Fully qualified methods

* A fully qualified method call is a way to explicitly specify which trait’s method you want to call, especially when:
  - Multiple traits define methods with the same name.
  - You’re calling a trait method that isn’t in scope via automatic dispatch.
  - You want to call a method on a trait for a type that implements it, but the method isn’t available via dot syntax.


In [23]:
mod FullyQualifiedSyntax {
    
    pub trait Drawable {
        fn draw(&self);
    }

    pub trait Deck {
        fn draw(&self) -> String;
    }

    pub struct DeckOfCards;

    impl Drawable for DeckOfCards {
        fn draw(&self) {
            println!("Drawing a deck of cards on the screen");
        }
    }

    impl Deck for DeckOfCards {
        fn draw(&self) -> String {
            println!("Drawing a card from the deck");
            "Ace of Spades".to_string()
        }
    }
}

use FullyQualifiedSyntax::*;

let deck_of_cards = DeckOfCards;

// deck_of_cards.draw(); // ambigous call of draw() method

FullyQualifiedSyntax::Drawable::draw(&deck_of_cards);
Deck::draw(&deck_of_cards);
<DeckOfCards as Deck>::draw(&deck_of_cards);

Drawing a deck of cards on the screen
Drawing a card from the deck
Drawing a card from the deck


## Reverse-Engineering Bounds

* When writing generic function or structs with trait bounds, it is often useful to reverse-engineer the bounds from the function body.
* This approach helps in understanding the minimum requirements for the types used in generics.
* Compiler errors can guide the addition of necessary trait bounds.

### First attempt

In [ ]:
fn dot<N>(v1: &[N], v2: &[N]) -> N {
    let mut total: N = N::default();
    for i in 0 .. v1.len() {
        total = total + v1[i] * v2[i];
    }
    total
}

### After reading compiler errors

In [ ]:
use std::ops::{Add, Mul};

fn dot<N: Default + Mul<Output=N> + Add<Output=N> + Copy>(v1: &[N], v2: &[N]) -> N 
{
    let mut total: N = N::default();
    for i in 0 .. v1.len() {
        total = total + v1[i] * v2[i];
    }
    total
}

In [ ]:
let v1 = [1, 2, 3];
let v2 = vec![4, 5, 6];

dot(&v1[..], &v2[..])

# Generics

## Generic Function

In [24]:
fn min<T: Ord>(a: T, b: T) -> T {
    if a < b {
        a
    } else {
        b
    }
}

In [25]:
min(1, 2)

1

In [26]:
min("hello", "world")

"hello"

## Generic Structure

* Generic structures allow us to define data types that can hold values of any type

In [27]:
struct Stack<T> {
    items: Vec<T>
}

impl<T> Stack<T> {
    fn push(&mut self, item: T) {
        self.items.push(item);
    }

    fn pop(&mut self) -> Option<T> {
        self.items.pop()
    }
}

fn main() {
    let mut stack = Stack{items: Vec::new()};
    stack.push(1);
    stack.push(2);
    stack.push(3);

    while let Some(item) = stack.pop() {
        println!("Popped: {}", item);
    }
}

main();

Popped: 3
Popped: 2
Popped: 1


## Generic Traits

* Traits can also be generic over types

In [28]:
mod explain {
    /// std::ops::Mul, the trait for types that support `*`.
    pub trait Mul<RHS=Self> {
        /// The resulting type after applying the `*` operator
        type Output;

        /// The method for the `*` operator
        fn mul(self, rhs: RHS) -> Self::Output;
    }
}

In [29]:
#[derive(Debug)]
struct Complex {
    real: f64,
    imag: f64
}

use std::ops::Mul;

impl Mul for Complex {
    type Output = Complex;

    fn mul(self, rhs: Complex) -> Complex {
        Complex {
            real: self.real * rhs.real - self.imag * rhs.imag,
            imag: self.real * rhs.imag + self.imag * rhs.real
        }
    }
}

fn main() {
    let a = Complex{real: 1.0, imag: 2.0};
    let b = Complex{real: 3.0, imag: 4.0};

    let c = a.mul(b);
    println!("Result: {} + {}i", c.real, c.imag);
}

main();

Result: -5 + 10i
